In [ ]:
!pip install transformers
!pip install pyngrok
!pip install flask

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from flask import Flask, request, jsonify
import os
import requests
from pyngrok import ngrok
# from flask_ngrok import run_with_ngrok

In [ ]:
ngrok.set_auth_token("2m3INDfD7mYEcHw8VB1STZuofFc_UBjx4mHCjjpu5iZbbxgN")

In [ ]:
app = Flask(__name__)


MODEL = None
TOKENIZER = None
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def clean_text(text):
    """
    Preprocess text using the same steps as training
    """
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation

    return text

def load_model(model_path):
    """
    Load the saved model and tokenizer
    """
    global MODEL, TOKENIZER

    # Load tokenizer from pretrained BERT
    TOKENIZER = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

    # Load the fine-tuned model
    MODEL = BertForSequenceClassification.from_pretrained(model_path)
    MODEL.to(DEVICE)
    MODEL.eval()

    print("Model loaded successfully!")

def predict_sentiment(text):
    """
    Predict sentiment for given text
    """
    # Clean the text first
    cleaned_text = clean_text(text)

    # Tokenize the cleaned text
    encoding = TOKENIZER(
        cleaned_text,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    # Move inputs to device
    input_ids = encoding['input_ids'].to(DEVICE)
    attention_mask = encoding['attention_mask'].to(DEVICE)

    # Get prediction
    with torch.no_grad():
        outputs = MODEL(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1)

    # Convert prediction to label
    sentiment = "positive" if prediction.item() == 1 else "negative"

    return sentiment

@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint for sentiment prediction
    """
    # Check if request contains JSON
    if not request.is_json:
        return jsonify({"error": "Request must be JSON"}), 400

    # Get the text from request
    data = request.get_json()
    if 'review_text' not in data:
        return jsonify({"error": "Missing review_text field"}), 400

    review_text = data['review_text']

    try:
        # Get prediction with cleaned text
        sentiment = predict_sentiment(review_text)

        # Return prediction
        return jsonify({
            "sentiment_prediction": sentiment
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    # Load model when starting the server
    model_path = "/content/drive/MyDrive/Assignment data science intern/Finetuned model"  # Replace with your model path
    load_model(model_path)
    ngrok.connect(5000)

    # Get the public URL
    tunnels = ngrok.get_tunnels()
    ngrok_url = tunnels[0].public_url
    print(f" * Public URL: {ngrok_url}")

    # Run Flask app
    app.run(port=5000)

Model loaded successfully!
 * Public URL: https://bd36-34-121-3-207.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:18:12] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:18:14] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:32] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:34] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:36] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:38] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:41] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:21:44] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:23:00] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Jan/2025 11:23:03] "POST /